## First Investigation of the Coudray Data

In [ ]:
# Import libraries
import openslide 
import openslide.deepzoom
import matplotlib.pyplot as plt
import numpy as np 

In [ ]:
# Read a svs file with openslide into a DeepZoomGenerator 

# here parameters like the number of levels and the downsampling factor can be specified
image = openslide.OpenSlide('/mnt/gaia/ICIDC/data_coudray/858bf199-5962-4d6e-a059-4d33b04edc17/TCGA-18-3415-01A-01-TS1.d922ab4a-e5c2-4b49-a173-964e569e3985.svs')

# here parameters like the tile size can be specified
image_slide = openslide.deepzoom.DeepZoomGenerator(image)

In [ ]:
# Print some information about the observed image
print(image.level_downsamples)
print(image_slide.level_count) # number of deepzoom levels for the slide
print(image_slide.tile_count) # number of tiles for the slide
print(image_slide.level_tiles) # (tiles_x, tiles_y) for each zoom level
print(image_slide.level_dimensions) # (pixel_x, pixel_y) for each zoom level
print(image_slide.get_dzi('png')) 
print(image_slide.get_tile(level=1, address=(0,0)))
print(image_slide.get_tile_coordinates(level=1, address=(0,0)))
print(image_slide.get_tile_dimensions(level=1, address=(0,0)))


In [ ]:
# Visualize a single tile 
tile = image_slide.get_tile(level=6, address=(0,0))
grey = tile.convert('L')
bw = grey.point(lambda x: 0 if x < 220 else 1, mode='F') # F -> 32-bit floating point values
bw_visualize = grey.point(lambda x: 0 if x < 220 else 1, mode='1') # 1 -> 1-bit pixels

plt.imshow(bw_visualize)
print('average background', np.average(np.array(np.asarray(bw_visualize))))

In [ ]:
print(np.array(np.asarray(bw)))
print(np.array(np.asarray(bw_visualize)))

## Some more snippets

How to create a generator for model training from a Dataset

In [ ]:
from data.data_set import Dataset
test_dataset = Dataset('/output_data/test_dataset.csv')
gen = test_dataset.get_generator(batch_size=8, shuffle=True, infinite=True)

for _ in range((len(test_dataset)//8) * 2):
    print(next(gen)[1])

How to visualize a patch

In [ ]:
from data.data_set import Dataset
test_dataset = Dataset('/output_data/test_dataset.csv')

_, (ax1, ax2) = plt.subplots(1,2, figsize=(16, 5))

# either direct access of data point
data_point = test_dataset.data_points[0]
ax1.imshow(data_point.get_patch())

# or from the generator

gen = test_dataset.get_generator()
patch = next(gen)[0]
ax2.imshow(patch[0,...]) # patch has batch dimension

In [ ]:
# to get a short dummy testset csv file by taking one (random) path from each patient using bash 
uniq -d -w 12 <csv_file> > <output_file>

Plotting the training loss from a history.json file

In [ ]:
import json
import matplotlib.pyplot as plt
with open(path, 'r') as json_file: 
    history = json.load(json_file)
    plt.plot(history.history['loss'], label='training loss')
    plt.plot(history.history['val_loss'], label='validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('epoch')
    plt.ylabel('cross entropy loss')
    plt.legend(loc='upper right')
    plt.show()